## Example of fir with two rotations

In [1]:
using Pkg; Pkg.activate("../.")
using Revise 

using LinearAlgebra, Statistics, Distributions 
using OrdinaryDiffEq
using SciMLSensitivity
using Optimization, OptimizationOptimisers, OptimizationOptimJL

using SphereFit

  Activating project at `~/.julia/dev/SphereFit`


In [2]:
using Random
rng = Random.default_rng()
Random.seed!(rng, 000666)
# Fisher concentration parameter on observations (small = more dispersion)
κ = 200 

200

Let's create a simple example consisting in two solid rotations around the globe with Fisher noise on top. 

In [3]:
# Total time simulation
tspan = [0, 130.0]
# Number of sample points
N_samples = 50
# Times where we sample points
times_samples = sort(rand(sampler(Uniform(tspan[1], tspan[2])), N_samples))

# Expected maximum angular deviation in one unit of time (degrees)
Δω₀ = 1.0   
# Angular velocity 
ω₀ = Δω₀ * π / 180.0
# Change point
τ₀ = 65.0
# Angular momentum
L0 = ω₀    .* [1.0, 0.0, 0.0]
L1 = 0.5ω₀ .* [0.0, sqrt(2), sqrt(2)]

# Solver tolerances 
reltol = 1e-7
abstol = 1e-7

1.0e-7

In [4]:
function true_rotation!(du, u, p, t)
    if t < τ₀
        L = p[1]
    else 
        L = p[2]
    end
    du .= cross(L, u)
end

prob = ODEProblem(true_rotation!, [0.0, 0.0, -1.0], tspan, [L0, L1])
true_sol  = solve(prob, Tsit5(), reltol=reltol, abstol=abstol, saveat=times_samples)

# Add Fisher noise to true solution 
X_noiseless = Array(true_sol)
X_true = mapslices(x -> rand(sampler(VonMisesFisher(x/norm(x), κ)), 1), X_noiseless, dims=1)

3×50 Matrix{Float64}:
 -0.0385817  -0.102203    0.0130063  …  -0.801308   -0.804544   -0.777151
  0.0229773   0.0624114   0.134454       0.593017    0.591397    0.625861
 -0.998991   -0.992804   -0.990834      -0.0789676  -0.0543948   0.065833

Let's make a plot of this using `PyCall` to call `cartopy` and `matplotlib`. 

In [5]:
X_true

3×50 Matrix{Float64}:
 -0.0385817  -0.102203    0.0130063  …  -0.801308   -0.804544   -0.777151
  0.0229773   0.0624114   0.134454       0.593017    0.591397    0.625861
 -0.998991   -0.992804   -0.990834      -0.0789676  -0.0543948   0.065833

In [6]:
X_true_sph = cart2sph(X_true, radians=false)

2×50 Matrix{Float64}:
 149.224   148.589    84.4747  104.938   …  143.496    143.681    141.155
 -87.4262  -83.1222  -82.2367  -78.8478      -4.52923   -3.11813    3.77468

### Python plots

In [26]:
using PyPlot, PyCall

mpl_colors = pyimport("matplotlib.colors")
mpl_colormap = pyimport("matplotlib.cm")
sns = pyimport("seaborn")
ccrs = pyimport("cartopy.crs")
feature = pyimport("cartopy.feature")

plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.Orthographic(central_latitude=-20, central_longitude=150))

# ax.coastlines()
ax.gridlines()
ax.set_global()

cmap = mpl_colormap.get_cmap("viridis")

sns.scatterplot(ax=ax, x = X_true_sph[:,1], y=X_true_sph[:,2], 
                # hue = df_data['time'], s=50,
                # palette="viridis",
                transform = ccrs.PlateCarree());

plt.savefig("testing.pdf", format="pdf")
# plt.show()